#### Dummy encoding Crime 2019 dataset

In [180]:
import pandas as pd
import numpy as np
import datetime as dt

In [275]:
dfcr = pd.read_csv(r'C:\Documents\projects\HackLA\311\data\WorkedonData\crime_data_2019c.csv', index_col=0)

In [276]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [277]:
dfcr.head(3)

,DR_NO,CreatedDate,DATE OCC,TIME OCC,PolicePrecinct,Rpt Dist No,Crm Cd,Crm Cd Desc,Vict Age,Vict Sex,Vict Descent,Status Desc,Premis Desc,Weapon Desc,Status Desc.1,LOCATION,LAT,LON
85876,190705631,2019-02-04,2010-07-01,1100,WILSHIRE,773,354,THEFT OF IDENTITY,60,F,B,Invest Cont,SINGLE FAMILY DWELLING,NaN,Invest Cont,1900 S RIDGELEY DR,34.0409,-118.3586
86769,191300738,2019-06-11,2010-06-07,1800,NEWTON,1383,745,VANDALISM - MISDEAMEANOR ($399 OR UNDER),23,M,B,Invest Cont,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",NaN,Invest Cont,5800 S BROADWAY,33.9829,-118.2783
87398,191226689,2019-11-11,2010-10-13,1500,77TH STREET,1242,354,THEFT OF IDENTITY,18,F,B,Invest Cont,SINGLE FAMILY DWELLING,NaN,Invest Cont,6500 3RD AV,33.9807,-118.3207


#### Drop columns which can't aggregate and it's too much info to hot encode

In [278]:
dfcr.drop(['DR_NO', 'Rpt Dist No', 'Crm Cd', 'LOCATION', 'LAT', 'LON'], axis=1, inplace=True)

In [279]:
dfcr.head(2)

,CreatedDate,DATE OCC,TIME OCC,PolicePrecinct,Crm Cd Desc,Vict Age,Vict Sex,Vict Descent,Status Desc,Premis Desc,Weapon Desc,Status Desc.1
85876,2019-02-04,2010-07-01,1100,WILSHIRE,THEFT OF IDENTITY,60,F,B,Invest Cont,SINGLE FAMILY DWELLING,NaN,Invest Cont
86769,2019-06-11,2010-06-07,1800,NEWTON,VANDALISM - MISDEAMEANOR ($399 OR UNDER),23,M,B,Invest Cont,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",NaN,Invest Cont


In [280]:
dfcr['DATE OCC'] = pd.to_datetime(dfcr['DATE OCC']).dt.normalize()

In [281]:
dfcr['CreatedDate'] = pd.to_datetime(dfcr['CreatedDate']).dt.normalize()

In [282]:
dfcr['CreatedDate'] = dfcr['CreatedDate'].dt.month


In [283]:
dfcr['DATE OCC'] = dfcr['DATE OCC'].dt.month

In [284]:
dfcr.rename(columns={'DATE OCC':'mnth_occ', 'CreatedDate':'month_created', 'TIME OCC':'hour'}, inplace=True)

In [285]:
dfcr.dtypes

month_created      int64
mnth_occ           int64
hour               int64
PolicePrecinct    object
Crm Cd Desc       object
Vict Age           int64
Vict Sex          object
Vict Descent      object
Status Desc       object
Premis Desc       object
Weapon Desc       object
Status Desc.1     object
dtype: object

In [286]:
dfcr.head(2)

,month_created,mnth_occ,hour,PolicePrecinct,Crm Cd Desc,Vict Age,Vict Sex,Vict Descent,Status Desc,Premis Desc,Weapon Desc,Status Desc.1
85876,2,7,1100,WILSHIRE,THEFT OF IDENTITY,60,F,B,Invest Cont,SINGLE FAMILY DWELLING,NaN,Invest Cont
86769,6,6,1800,NEWTON,VANDALISM - MISDEAMEANOR ($399 OR UNDER),23,M,B,Invest Cont,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",NaN,Invest Cont


#### Drop rows where Vict Sex = H or N

In [287]:
dfcr['Vict Sex'].value_counts()

M    97642
F    87958
X    15027
H        6
N        1
Name: Vict Sex, dtype: int64

In [288]:
dfcr.drop(dfcr.loc[dfcr['Vict Sex']=='H'].index, inplace=True)

In [289]:
dfcr.drop(dfcr.loc[dfcr['Vict Sex']=='N'].index, inplace=True)

In [290]:
dfcr['Vict Sex'].value_counts()

M    97642
F    87958
X    15027
Name: Vict Sex, dtype: int64

In [291]:
dfcr.head(2)

,month_created,mnth_occ,hour,PolicePrecinct,Crm Cd Desc,Vict Age,Vict Sex,Vict Descent,Status Desc,Premis Desc,Weapon Desc,Status Desc.1
85876,2,7,1100,WILSHIRE,THEFT OF IDENTITY,60,F,B,Invest Cont,SINGLE FAMILY DWELLING,NaN,Invest Cont
86769,6,6,1800,NEWTON,VANDALISM - MISDEAMEANOR ($399 OR UNDER),23,M,B,Invest Cont,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",NaN,Invest Cont


### categorize Vict Age & Time OCC, look and see if need to clean up Vict Descent and Weapon Desc

#### Create categories for age

In [292]:
dfcr.drop(dfcr.loc[dfcr['Vict Age'] < 1].index, inplace=True)

In [293]:
dfcr['Vict Age'].value_counts().sort_index().tail()

95     30
96     19
97     18
98     18
99    121
Name: Vict Age, dtype: int64

In [294]:
dfcr.shape

(178542, 12)

In [295]:
#drop age 99 - amt much higher than surrounding numbers - seems like a place holder
dfcr.drop(dfcr.loc[dfcr['Vict Age']>=99].index, inplace=True)

In [296]:
dfcr.shape

(178421, 12)

In [297]:
dfcr.loc[dfcr['Vict Age'] <= 12, 'vic_age'] = 'under13'
dfcr.loc[(dfcr['Vict Age'] >= 13) & (dfcr['Vict Age'] <=25),  'vic_age'] = '13-25'
dfcr.loc[(dfcr['Vict Age'] >= 26) & (dfcr['Vict Age'] <=40),  'vic_age'] = '26-40'
dfcr.loc[(dfcr['Vict Age'] >= 41) & (dfcr['Vict Age'] <=60),  'vic_age'] = '41-60'
dfcr.loc[(dfcr['Vict Age'] >= 61) & (dfcr['Vict Age'] <=80),  'vic_age'] = '61-80'
dfcr.loc[dfcr['Vict Age'] > 80, 'vic_age'] = 'over80'


In [298]:
dfcr.head(3)

,month_created,mnth_occ,hour,PolicePrecinct,Crm Cd Desc,Vict Age,Vict Sex,Vict Descent,Status Desc,Premis Desc,Weapon Desc,Status Desc.1,vic_age
85876,2,7,1100,WILSHIRE,THEFT OF IDENTITY,60,F,B,Invest Cont,SINGLE FAMILY DWELLING,NaN,Invest Cont,41-60
86769,6,6,1800,NEWTON,VANDALISM - MISDEAMEANOR ($399 OR UNDER),23,M,B,Invest Cont,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",NaN,Invest Cont,13-25
87398,11,10,1500,77TH STREET,THEFT OF IDENTITY,18,F,B,Invest Cont,SINGLE FAMILY DWELLING,NaN,Invest Cont,13-25


In [299]:
dfcr.to_csv(r'C:\Documents\projects\HackLA\311\data\WorkedonData\crime_data_2019c.csv')

#### Create categories for 'hour'(formerly known as 'TIME OCC') in a new column 'time' and delete 'hour'

In [301]:
dfcr['hour'].dtype

dtype('int64')

In [358]:

dfcr.loc[(dfcr['hour'] >= 500) & (dfcr['hour'] <=900),  'time'] = '5am-9am'
dfcr.loc[(dfcr['hour'] >= 1700) & (dfcr['hour'] < 2000),  'time'] = '5pm-8pm'
dfcr.loc[(dfcr['hour'] >= 1500) & (dfcr['hour'] < 1700),  'time'] = '3pm-5pm'
dfcr.loc[(dfcr['hour'] >= 1) & (dfcr['hour'] < 3),  'time'] = '9am-3pm'
dfcr.loc[(dfcr['hour'] >= 200) & (dfcr['hour'] < 500),  'time'] = '2am-5am'
dfcr.loc[(dfcr['hour'] >= 5) & (dfcr['hour'] < 8),  'time'] = '5pm-8pm'
dfcr.loc[(dfcr['hour'] >= 100) & (dfcr['hour'] < 200),  'time'] = 'mid-2am'
dfcr.loc[(dfcr['hour'] >= 1200) & (dfcr['hour'] < 1500),  'time'] = '9am-3pm'
dfcr.loc[(dfcr['hour'] >= 2000) & (dfcr['hour'] <=2400),  'time'] = '8pm-mid'
dfcr.loc[(dfcr['hour'] > 0) & (dfcr['hour'] <= 100),  'time'] = 'mid-2am'

In [361]:
dfcr.head()

,month_created,mnth_occ,hour,PolicePrecinct,Crm Cd Desc,Vict Age,Vict Sex,Vict Descent,Status Desc,Premis Desc,Weapon Desc,Status Desc.1,vic_age,time
85876,2,7,1100,WILSHIRE,THEFT OF IDENTITY,60,F,B,Invest Cont,SINGLE FAMILY DWELLING,NaN,Invest Cont,41-60,8pm-mid
86769,6,6,1800,NEWTON,VANDALISM - MISDEAMEANOR ($399 OR UNDER),23,M,B,Invest Cont,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",NaN,Invest Cont,13-25,5pm-8pm
87398,11,10,1500,77TH STREET,THEFT OF IDENTITY,18,F,B,Invest Cont,SINGLE FAMILY DWELLING,NaN,Invest Cont,13-25,3pm-5pm
87436,9,1,800,HARBOR,"THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LI...",50,M,H,Invest Cont,SINGLE FAMILY DWELLING,NaN,Invest Cont,41-60,5am-9am
87649,3,1,1,VAN NUYS,INTIMATE PARTNER - SIMPLE ASSAULT,34,F,A,Invest Cont,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)","STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",Invest Cont,26-40,mid-2am


In [364]:
dfcr.drop(['hour'], axis=1, inplace=True)

In [365]:
dfcr.head()

,month_created,mnth_occ,PolicePrecinct,Crm Cd Desc,Vict Age,Vict Sex,Vict Descent,Status Desc,Premis Desc,Weapon Desc,Status Desc.1,vic_age,time
85876,2,7,WILSHIRE,THEFT OF IDENTITY,60,F,B,Invest Cont,SINGLE FAMILY DWELLING,NaN,Invest Cont,41-60,8pm-mid
86769,6,6,NEWTON,VANDALISM - MISDEAMEANOR ($399 OR UNDER),23,M,B,Invest Cont,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",NaN,Invest Cont,13-25,5pm-8pm
87398,11,10,77TH STREET,THEFT OF IDENTITY,18,F,B,Invest Cont,SINGLE FAMILY DWELLING,NaN,Invest Cont,13-25,3pm-5pm
87436,9,1,HARBOR,"THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LI...",50,M,H,Invest Cont,SINGLE FAMILY DWELLING,NaN,Invest Cont,41-60,5am-9am
87649,3,1,VAN NUYS,INTIMATE PARTNER - SIMPLE ASSAULT,34,F,A,Invest Cont,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)","STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",Invest Cont,26-40,mid-2am


In [366]:
dfcr.to_csv(r'C:\Documents\projects\HackLA\311\data\WorkedonData\crime_data_2019d.csv')

### Crime Description

In [375]:
dfcr['Crm Cd Desc'].value_counts()

BATTERY - SIMPLE ASSAULT                                    18732
BURGLARY FROM VEHICLE                                       16382
THEFT PLAIN - PETTY ($950 & UNDER)                          13670
INTIMATE PARTNER - SIMPLE ASSAULT                           12032
ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT              10496
VANDALISM - FELONY ($400 & OVER, ALL CHURCH VANDALISMS)     10349
BURGLARY                                                    10110
THEFT OF IDENTITY                                            9720
ROBBERY                                                      7082
THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LIVESTK,PROD     6731
VANDALISM - MISDEAMEANOR ($399 OR UNDER)                     6411
THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER)              5935
CRIMINAL THREATS - NO WEAPON DISPLAYED                       4838
THEFT FROM MOTOR VEHICLE - GRAND ($950.01 AND OVER)          3749
INTIMATE PARTNER - AGGRAVATED ASSAULT                        2915
BRANDISH W

In [380]:
dfcr['Crm Cd Desc'].value_counts().tail(20)

PICKPOCKET, ATTEMPT                                         3
DISRUPT SCHOOL                                              3
PURSE SNATCHING - ATTEMPT                                   3
TELEPHONE PROPERTY - DAMAGE                                 2
TILL TAP - GRAND THEFT ($950.01 & OVER)                     2
PETTY THEFT - AUTO REPAIR                                   2
CONSPIRACY                                                  2
GRAND THEFT / INSURANCE FRAUD                               2
INCEST (SEXUAL ACTS BETWEEN BLOOD RELATIVES)                2
BRIBERY                                                     2
BIGAMY                                                      2
THEFT, COIN MACHINE - GRAND ($950.01 & OVER)                1
ABORTION/ILLEGAL                                            1
GRAND THEFT / AUTO REPAIR                                   1
THEFT, COIN MACHINE - ATTEMPT                               1
DISHONEST EMPLOYEE ATTEMPTED THEFT                          1
TILL TAP

#### Delete crimes that occur less than 5 times so have less columns when create dummies

In [391]:
test = dfcr
test['Crm Cd Desc'].value_counts().tail(25)

DRUNK ROLL                                                  6
BIKE - ATTEMPTED STOLEN                                     5
DOCUMENT WORTHLESS ($200 & UNDER)                           5
DISHONEST EMPLOYEE - PETTY THEFT                            5
DISHONEST EMPLOYEE - GRAND THEFT                            4
PICKPOCKET, ATTEMPT                                         3
DISRUPT SCHOOL                                              3
PURSE SNATCHING - ATTEMPT                                   3
TELEPHONE PROPERTY - DAMAGE                                 2
TILL TAP - GRAND THEFT ($950.01 & OVER)                     2
PETTY THEFT - AUTO REPAIR                                   2
CONSPIRACY                                                  2
GRAND THEFT / INSURANCE FRAUD                               2
INCEST (SEXUAL ACTS BETWEEN BLOOD RELATIVES)                2
BRIBERY                                                     2
BIGAMY                                                      2
THEFT, C

In [399]:
tstrst2 = test[test.groupby('Crm Cd Desc')['Crm Cd Desc'].transform('size') > 5]



In [406]:
100 - 100 * float(tstrst2.shape[0])/float(test.shape[0])

0.02970502351180926

##### lose less than 1% when delete rows where count of 'Crm Cd Desc' is 5 or less


In [407]:
dfcr = dfcr[dfcr.groupby('Crm Cd Desc')['Crm Cd Desc'].transform('size') > 5]

In [411]:
dfcr['Crm Cd Desc'].value_counts().tail()

CHILD ABANDONMENT                                           10
WEAPONS POSSESSION/BOMBING                                   9
DRUGS, TO A MINOR                                            7
DRUNK ROLL                                                   6
REPLICA FIREARMS(SALE,DISPLAY,MANUFACTURE OR DISTRIBUTE)     6
Name: Crm Cd Desc, dtype: int64

### Create Dummies

In [412]:
dfcr.head(2)

,month_created,mnth_occ,PolicePrecinct,Crm Cd Desc,Vict Age,Vict Sex,Vict Descent,Status Desc,Premis Desc,Weapon Desc,Status Desc.1,vic_age,time
85876,2,7,WILSHIRE,THEFT OF IDENTITY,60,F,B,Invest Cont,SINGLE FAMILY DWELLING,NaN,Invest Cont,41-60,8pm-mid
86769,6,6,NEWTON,VANDALISM - MISDEAMEANOR ($399 OR UNDER),23,M,B,Invest Cont,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",NaN,Invest Cont,13-25,5pm-8pm


Create small subset so can understand what's happening
Try creating dummies separately with separate prefixes so can create for loops when group them or 
use this code: pd.get_dummies(dfsub, columns=['Crm Cd Desc']).groupby(['CreatedDate', 'PolicePrecinct'], as_index=False).sum()

In [414]:
dfsub = dfcr.iloc[0:10,[0,2,3]]
dfsub

,month_created,PolicePrecinct,Crm Cd Desc
85876,2,WILSHIRE,THEFT OF IDENTITY
86769,6,NEWTON,VANDALISM - MISDEAMEANOR ($399 OR UNDER)
87398,11,77TH STREET,THEFT OF IDENTITY
87436,9,HARBOR,"THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LI..."
87649,3,VAN NUYS,INTIMATE PARTNER - SIMPLE ASSAULT
88334,9,OLYMPIC,THROWING OBJECT AT MOVING VEHICLE
89516,5,WILSHIRE,THEFT OF IDENTITY
89789,12,MISSION,CRM AGNST CHLD (13 OR UNDER) (14-15 & SUSP 10 ...
90564,4,MISSION,THEFT OF IDENTITY
91522,3,HARBOR,"RAPE, FORCIBLE"


In [415]:
pd.get_dummies(dfsub, columns=['Crm Cd Desc']).groupby(['month_created', 'PolicePrecinct'], as_index=False).sum()

,month_created,PolicePrecinct,Crm Cd Desc_CRM AGNST CHLD (13 OR UNDER) (14-15 & SUSP 10 YRS OLDER),Crm Cd Desc_INTIMATE PARTNER - SIMPLE ASSAULT,"Crm Cd Desc_RAPE, FORCIBLE",Crm Cd Desc_THEFT OF IDENTITY,"Crm Cd Desc_THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LIVESTK,PROD",Crm Cd Desc_THROWING OBJECT AT MOVING VEHICLE,Crm Cd Desc_VANDALISM - MISDEAMEANOR ($399 OR UNDER)
0,2,WILSHIRE,0,0,0,1,0,0,0
1,3,HARBOR,0,0,1,0,0,0,0
2,3,VAN NUYS,0,1,0,0,0,0,0
3,4,MISSION,0,0,0,1,0,0,0
4,5,WILSHIRE,0,0,0,1,0,0,0
5,6,NEWTON,0,0,0,0,0,0,1
6,9,HARBOR,0,0,0,0,1,0,0
7,9,OLYMPIC,0,0,0,0,0,1,0
8,11,77TH STREET,0,0,0,1,0,0,0
9,12,MISSION,1,0,0,0,0,0,0


This code works.  Let's expand it.

In [416]:
dfcr.head(2)

,month_created,mnth_occ,PolicePrecinct,Crm Cd Desc,Vict Age,Vict Sex,Vict Descent,Status Desc,Premis Desc,Weapon Desc,Status Desc.1,vic_age,time
85876,2,7,WILSHIRE,THEFT OF IDENTITY,60,F,B,Invest Cont,SINGLE FAMILY DWELLING,NaN,Invest Cont,41-60,8pm-mid
86769,6,6,NEWTON,VANDALISM - MISDEAMEANOR ($399 OR UNDER),23,M,B,Invest Cont,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",NaN,Invest Cont,13-25,5pm-8pm


In [424]:
dfcr_d = dfcr

In [425]:
dfcr_d.head()

,month_created,mnth_occ,PolicePrecinct,Crm Cd Desc,Vict Age,Vict Sex,Vict Descent,Status Desc,Premis Desc,Weapon Desc,Status Desc.1,vic_age,time
85876,2,7,WILSHIRE,THEFT OF IDENTITY,60,F,B,Invest Cont,SINGLE FAMILY DWELLING,NaN,Invest Cont,41-60,8pm-mid
86769,6,6,NEWTON,VANDALISM - MISDEAMEANOR ($399 OR UNDER),23,M,B,Invest Cont,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",NaN,Invest Cont,13-25,5pm-8pm
87398,11,10,77TH STREET,THEFT OF IDENTITY,18,F,B,Invest Cont,SINGLE FAMILY DWELLING,NaN,Invest Cont,13-25,3pm-5pm
87436,9,1,HARBOR,"THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LI...",50,M,H,Invest Cont,SINGLE FAMILY DWELLING,NaN,Invest Cont,41-60,5am-9am
87649,3,1,VAN NUYS,INTIMATE PARTNER - SIMPLE ASSAULT,34,F,A,Invest Cont,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)","STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",Invest Cont,26-40,mid-2am


In [426]:
dfcr_d.drop(['mnth_occ', 'Vict Age'], axis=1, inplace=True )

C:\Users\lizge\AppData\Local\Temp\ipykernel_3452\1686615780.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfcr_d.drop(['mnth_occ', 'Vict Age'], axis=1, inplace=True )


In [428]:
dfcr_d.drop(['Status Desc.1'], axis=1, inplace=True )

C:\Users\lizge\AppData\Local\Temp\ipykernel_3452\3985823387.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfcr_d.drop(['Status Desc.1'], axis=1, inplace=True )


In [429]:
dfcr_d.head(2)

,month_created,PolicePrecinct,Crm Cd Desc,Vict Sex,Vict Descent,Status Desc,Premis Desc,Weapon Desc,vic_age,time
85876,2,WILSHIRE,THEFT OF IDENTITY,F,B,Invest Cont,SINGLE FAMILY DWELLING,NaN,41-60,8pm-mid
86769,6,NEWTON,VANDALISM - MISDEAMEANOR ($399 OR UNDER),M,B,Invest Cont,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",NaN,13-25,5pm-8pm


In [430]:
dfcr_d = pd.get_dummies(dfcr_d, columns=['Crm Cd Desc', 'Vict Sex', 'Status Desc', 'Premis Desc', 'Weapon Desc', 
    'vic_age', 'time']).groupby(['month_created', 'PolicePrecinct'], as_index=False).sum()

dfcr_d.head()

C:\Users\lizge\AppData\Local\Temp\ipykernel_3452\1229381680.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  'vic_age', 'time']).groupby(['month_created', 'PolicePrecinct'], as_index=False).sum()


,month_created,PolicePrecinct,Crm Cd Desc_ARSON,Crm Cd Desc_ASSAULT WITH DEADLY WEAPON ON POLICE OFFICER,"Crm Cd Desc_ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",Crm Cd Desc_ATTEMPTED ROBBERY,Crm Cd Desc_BATTERY - SIMPLE ASSAULT,Crm Cd Desc_BATTERY ON A FIREFIGHTER,Crm Cd Desc_BATTERY POLICE (SIMPLE),Crm Cd Desc_BATTERY WITH SEXUAL CONTACT,Crm Cd Desc_BIKE - STOLEN,Crm Cd Desc_BOMB SCARE,Crm Cd Desc_BRANDISH WEAPON,"Crm Cd Desc_BUNCO, ATTEMPT","Crm Cd Desc_BUNCO, GRAND THEFT","Crm Cd Desc_BUNCO, PETTY THEFT",Crm Cd Desc_BURGLARY,Crm Cd Desc_BURGLARY FROM VEHICLE,"Crm Cd Desc_BURGLARY FROM VEHICLE, ATTEMPTED","Crm Cd Desc_BURGLARY, ATTEMPTED",Crm Cd Desc_CHILD ABANDONMENT,Crm Cd Desc_CHILD ABUSE (PHYSICAL) - AGGRAVATED ASSAULT,Crm Cd Desc_CHILD ABUSE (PHYSICAL) - SIMPLE ASSAULT,Crm Cd Desc_CHILD ANNOYING (17YRS & UNDER),Crm Cd Desc_CHILD NEGLECT (SEE 300 W.I.C.),Crm Cd Desc_CHILD PORNOGRAPHY,Crm Cd Desc_CHILD STEALING,Crm Cd Desc_CONTEMPT OF COURT,Crm Cd Desc_CONTRIBUTING,Crm Cd Desc_COUNTERFEIT,"Crm Cd Desc_CREDIT CARDS, FRAUD USE ($950 & UNDER","Crm Cd Desc_CREDIT CARDS, FRAUD USE ($950.01 & OVER)",Crm Cd Desc_CRIMINAL HOMICIDE,Crm Cd Desc_CRIMINAL THREATS - NO WEAPON DISPLAYED,Crm Cd Desc_CRM AGNST CHLD (13 OR UNDER) (14-15 & SUSP 10 YRS OLDER),Crm Cd Desc_CRUELTY TO ANIMALS,"Crm Cd Desc_DEFRAUDING INNKEEPER/THEFT OF SERVICES, $950 & UNDER","Crm Cd Desc_DEFRAUDING INNKEEPER/THEFT OF SERVICES, OVER $950.01",Crm Cd Desc_DISCHARGE FIREARMS/SHOTS FIRED,Crm Cd Desc_DISTURBING THE PEACE,Crm Cd Desc_DOCUMENT FORGERY / STOLEN FELONY,Crm Cd Desc_DOCUMENT WORTHLESS ($200.01 & OVER),Crm Cd Desc_DRIVING WITHOUT OWNER CONSENT (DWOC),"Crm Cd Desc_DRUGS, TO A MINOR",Crm Cd Desc_DRUNK ROLL,"Crm Cd Desc_EMBEZZLEMENT, GRAND THEFT ($950.01 & OVER)","Crm Cd Desc_EMBEZZLEMENT, PETTY THEFT ($950 & UNDER)",Crm Cd Desc_EXTORTION,Crm Cd Desc_FAILURE TO YIELD,Crm Cd Desc_FALSE IMPRISONMENT,Crm Cd Desc_FALSE POLICE REPORT,Crm Cd Desc_HUMAN TRAFFICKING - COMMERCIAL SEX ACTS,Crm Cd Desc_HUMAN TRAFFICKING - INVOLUNTARY SERVITUDE,Crm Cd Desc_ILLEGAL DUMPING,Crm Cd Desc_INDECENT EXPOSURE,Crm Cd Desc_INTIMATE PARTNER - AGGRAVATED ASSAULT,Crm Cd Desc_INTIMATE PARTNER - SIMPLE ASSAULT,Crm Cd Desc_KIDNAPPING,Crm Cd Desc_KIDNAPPING - GRAND ATTEMPT,"Crm Cd Desc_LETTERS, LEWD - TELEPHONE CALLS, LEWD",Crm Cd Desc_LEWD CONDUCT,Crm Cd Desc_LEWD/LASCIVIOUS ACTS WITH CHILD,Crm Cd Desc_ORAL COPULATION,Crm Cd Desc_OTHER ASSAULT,Crm Cd Desc_OTHER MISCELLANEOUS CRIME,Crm Cd Desc_PANDERING,Crm Cd Desc_PEEPING TOM,Crm Cd Desc_PICKPOCKET,Crm Cd Desc_PIMPING,Crm Cd Desc_PROWLER,Crm Cd Desc_PURSE SNATCHING,"Crm Cd Desc_RAPE, ATTEMPTED","Crm Cd Desc_RAPE, FORCIBLE",Crm Cd Desc_RECKLESS DRIVING,"Crm Cd Desc_REPLICA FIREARMS(SALE,DISPLAY,MANUFACTURE OR DISTRIBUTE)",Crm Cd Desc_RESISTING ARREST,Crm Cd Desc_ROBBERY,Crm Cd Desc_SEX OFFENDER REGISTRANT OUT OF COMPLIANCE,"Crm Cd Desc_SEX,UNLAWFUL(INC MUTUAL CONSENT, PENETRATION W/ FRGN OBJ",Crm Cd Desc_SEXUAL PENETRATION W/FOREIGN OBJECT,Crm Cd Desc_SHOPLIFTING - ATTEMPT,Crm Cd Desc_SHOPLIFTING - PETTY THEFT ($950 & UNDER),Crm Cd Desc_SHOPLIFTING-GRAND THEFT ($950.01 & OVER),Crm Cd Desc_SHOTS FIRED AT INHABITED DWELLING,"Crm Cd Desc_SHOTS FIRED AT MOVING VEHICLE, TRAIN OR AIRCRAFT",Crm Cd Desc_SODOMY/SEXUAL CONTACT B/W PENIS OF ONE PERS TO ANUS OTH,Crm Cd Desc_STALKING,Crm Cd Desc_THEFT FROM MOTOR VEHICLE - ATTEMPT,Crm Cd Desc_THEFT FROM MOTOR VEHICLE - GRAND ($950.01 AND OVER),Crm Cd Desc_THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER),Crm Cd Desc_THEFT FROM PERSON - ATTEMPT,Crm Cd Desc_THEFT OF IDENTITY,Crm Cd Desc_THEFT PLAIN - ATTEMPT,Crm Cd Desc_THEFT PLAIN - PETTY ($950 & UNDER),"Crm Cd Desc_THEFT, COIN MACHINE - PETTY ($950 & UNDER)","Crm Cd Desc_THEFT, PERSON","Crm Cd Desc_THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LIVESTK,PROD",Crm Cd Desc_THREATENING PHONE CALLS/LETTERS,Crm Cd Desc_THROWING OBJECT AT MOVING VEHICLE,Crm Cd Desc_TRESPASSING,Crm Cd Desc_UNAUTHORIZED COMPUTER ACCESS,"Crm Cd Desc_VANDALISM - FELONY ($400 & O

In [432]:
dfcr_d.shape

(252, 495)

In [433]:
dfcr_d.to_csv(r'C:\Documents\projects\HackLA\311\data\WorkedonData\cr_2019_grpby_dummies.csv')